# Seq2Seq architectures

Önceki bölümde tartışılan CTC modelleri, dönüştürücü mimarisinin yalnızca kodlayıcı kısmını kullanmıştır. Bir kodlayıcı-kod çözücü modeli oluşturmak için kod çözücüyü de eklediğimizde, bu bir diziden diziye model veya kısaca seq2seq olarak adlandırılır. Model, bir tür veriden oluşan bir diziyi başka bir tür veriden oluşan bir diziye eşler.

Yalnızca kodlayıcı dönüştürücü modellerinde, kodlayıcı giriş dizisindeki her bir öğe için bir tahmin yapar. Bu nedenle, hem giriş hem de çıkış dizileri her zaman aynı uzunlukta olacaktır. Wav2Vec2 gibi CTC modellerinde giriş dalga biçimi önce alt örneklenirdi, ancak yine de her 20 ms'lik ses için bir tahmin vardı.

Bir seq2seq modelinde, böyle bire bir yazışma yoktur ve giriş ve çıkış dizileri farklı uzunluklara sahip olabilir. Bu da seq2seq modellerini metin özetleme veya farklı diller arasında çeviri gibi NLP görevlerinin yanı sıra konuşma tanıma gibi ses görevleri için de uygun hale getirir.

Kod çözücünün mimarisi kodlayıcıya çok benzer ve her ikisi de ana özellik olarak öz dikkat ile benzer katmanlar kullanır. Ancak kod çözücü, kodlayıcıdan farklı bir görevi yerine getirir. Bunun nasıl çalıştığını görmek için, bir seq2seq modelinin otomatik konuşma tanımayı nasıl yapabileceğini inceleyelim.

## Automatic speech recognition

Whisper'ın mimarisi aşağıdaki gibidir (şekil OpenAI Whisper blogunun izniyle):

![](https://huggingface.co/blog/assets/111_fine_tune_whisper/whisper_architecture.svg)

Bu oldukça tanıdık gelecektir. Sol tarafta transformatör kodlayıcı yer almaktadır. Bu, **girdi olarak bir log-mel spektrogramı alır** ve konuşmadan önemli özellikleri çıkaran bir kodlayıcı gizli durumlar dizisi oluşturmak için bu spektrogramı kodlar. Bu gizli durumlar tensörü, girdi dizisini bir bütün olarak temsil eder ve girdi konuşmasının "anlamını" etkili bir şekilde kodlar.

Kodlayıcının çıktısı daha sonra **çapraz dikkat** adı verilen bir mekanizma kullanılarak sağda gösterilen dönüştürücü kod çözücüye aktarılır. Bu kendi kendine dikkat etme gibidir ancak kodlayıcı çıkışına dikkat eder. Bu noktadan sonra kodlayıcıya artık ihtiyaç yoktur.

Kod çözücü, içinde sadece bir "başlangıç" belirteci (Whisper durumunda **SOT**) bulunan bir başlangıç dizisinden başlayarak, **her seferinde tek bir belirteç olmak üzere, otoregresif bir şekilde bir dizi metin belirtecini tahmin eder**. **Takip eden her zaman adımında, bir önceki çıkış dizisi yeni giriş dizisi olarak kod çözücüye geri beslenir**. Bu şekilde, kod çözücü her seferinde bir yeni belirteç yayar ve bir "son" belirteci tahmin edene veya maksimum zaman adımı sayısına ulaşılana kadar çıktı dizisini sürekli olarak büyütür.

Kod çözücünün mimarisi çoğunlukla kodlayıcı ile aynı olsa da, iki büyük fark vardır:

- kod çözücü, **kodlayıcının giriş dizisinin temsiline bakmasına izin veren bir çapraz dikkat mekanizmasına sahiptir**
- kod çözücünün dikkati nedenseldir - **kod çözücünün geleceğe bakmasına izin verilmez**.

Bu tasarımda kod çözücü, kodlayıcıdan gelen gizli durum temsillerini işleyen ve karşılık gelen metin transkripsiyonlarını üreten bir dil modeli rolünü oynar. CTC modeli harici bir dil modeliyle birleştirilse bile, seq2seq sistemi aynı eğitim verileri ve kayıp fonksiyonuyla uçtan uca eğitilebildiğinden, daha fazla esneklik ve genellikle daha üstün performans sağladığından, bu CTC'den daha güçlü bir yaklaşımdır.

Bir seq2seq ASR modeli için tipik bir kayıp fonksiyonu, modelin son katmanı olası belirteçler üzerinde bir olasılık dağılımı öngördüğü için **çapraz entropi kaybıdır**. Bu genellikle son diziyi oluşturmak için ışın arama gibi tekniklerle birleştirilir. Konuşma tanıma metriği, tahmin edilen metni hedef metne dönüştürmek için kaç tane ikame, ekleme ve çıkarmanın gerekli olduğunu ölçen **WER** veya **kelime hata oranıdır - ne kadar az olursa, puan o kadar iyi olur**.

## Text-to-speech

Bu sizi şaşırtmayabilir: TTS için bir seq2seq modeli temelde yukarıda açıklananla aynı şekilde çalışır, ancak girişler ve çıkışlar değiştirilir! Dönüştürücü kodlayıcı bir dizi metin simgesini alır ve giriş metnini temsil eden bir dizi gizli durum çıkarır. **Transformer kod çözücü, kodlayıcı çıktısına çapraz dikkat uygular ve bir spektrogram tahmin eder**.

ASR modeli ile kod çözücü, içinde sadece özel "başlangıç" belirteci bulunan bir dizi kullanılarak başlatılmıştır. TTS modeli için, kod çözme işlemine "başlangıç belirteci" olarak işlev gören ve **tamamı sıfır olan bir uzunluğunda bir spektrogram ile başlayabiliriz**. Bu başlangıç spektrogramı ve kodlayıcının gizli durum temsilleri üzerindeki çapraz dikkatler göz önüne alındığında, kod çözücü daha sonra bu spektrogram için bir sonraki zaman dilimini tahmin eder ve spektrogramı her seferinde bir zaman adımında istikrarlı bir şekilde büyütür.

![](https://huggingface.co/datasets/huggingface-course/audio-course-images/resolve/main/speecht5_decoding.png)

Peki kod çözücü ne zaman duracağını nasıl bilir? SpeechT5 modelinde bu, kod çözücünün ikinci bir diziyi tahmin etmesini sağlayarak ele alınır. Bu, mevcut zaman adımının son zaman adımı olma olasılığını içerir. Çıkarım zamanında ses üretilirken, bu olasılık belirli bir eşiğin üzerindeyse (örneğin 0,5), kod çözücü spektrogramın bittiğini ve üretim döngüsünün sona ermesi gerektiğini belirtir.

Kod çözme işlemi bittikten ve spektrogramı içeren bir çıktı dizisine sahip olduktan sonra **SpeechT5, spektrogramı iyileştirmek için birkaç konvolüsyon katmanından oluşan bir `post-net` kullanır**.

TTS modelinin eğitimi sırasında **hedefler de spektrogramlardır ve kayıp L1 veya MSE'dir**. Çıkarım zamanında, çıkış spektrogramını bir ses dalga şekline dönüştürmek istiyoruz, böylece onu gerçekten dinleyebiliyoruz. Bunun için harici bir model olan **vocoder** kullanılır. Bu vokoder seq2seq mimarisinin bir parçası değildir ve ayrı olarak eğitilir.

TTS'yi zorlaştıran şey, bire çok eşleme olmasıdır. **Konuşmadan metne ile giriş konuşmasına karşılık gelen tek bir doğru çıkış metni vardır, ancak metinden konuşmaya ile giriş metni birçok olası konuşma sesine eşlenebilir**. Örneğin, farklı konuşmacılar cümlenin farklı kısımlarını vurgulamayı seçebilir. Bu da TTS modellerinin değerlendirilmesini zorlaştırır. Bu nedenle, **L1 veya MSE kayıp değeri aslında çok anlamlı değildir - aynı metni bir spektrogramda temsil etmenin birden fazla yolu vardır. Bu nedenle TTS modelleri genellikle MOS veya ortalama görüş puanı olarak bilinen bir metrik kullanılarak insan dinleyiciler tarafından değerlendirilir**.

## Conclusion

Seq2seq yaklaşımı, yalnızca kodlayıcı modelinden daha güçlüdür. Giriş dizisinin kodlanmasını çıkış dizisinin kod çözme işleminden ayırarak, ses ve metnin hizalanması daha az sorun teşkil eder.

Bununla birlikte, kod çözme işlemi bir kerede değil, her seferinde bir adımda gerçekleştiğinden, kodlayıcı-kod çözücü modeli de daha yavaştır. Sekans ne kadar uzun olursa tahmin de o kadar yavaş olur. Otoregresif modeller de tekrarlara takılabilir veya kelimeleri atlayabilir. Demet arama gibi teknikler tahminlerin kalitesini artırmaya yardımcı olabilir, ancak aynı zamanda kod çözme işlemini daha da yavaşlatır.